# Customer Lifetime Value Prediction

In [1]:
from __future__ import division
from datetime import datetime, timedelta,date
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import plotly.offline as pyoff
import plotly.graph_objs as go

from sklearn.cluster import KMeans
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import classification_report,confusion_matrix

pyoff.init_notebook_mode()

In [2]:
df_data = pd.read_csv('../datasets/OnlineRetail.csv',encoding = 'unicode_escape')

In [3]:
df_data['InvoiceDate'] = pd.to_datetime(df_data['InvoiceDate'])
df_uk = df_data.query("Country=='United Kingdom'").reset_index(drop=True)

In [4]:
df_data.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom

In [5]:
df_3m = df_uk[(df_uk.InvoiceDate < datetime(2011,6,1)) & (df_uk.InvoiceDate >= datetime(2011,3,1))].reset_index(drop=True)
df_6m = df_uk[(df_uk.InvoiceDate >= datetime(2011,6,1)) & (df_uk.InvoiceDate < datetime(2011,12,1))].reset_index(drop=True)

In [6]:
df_user = pd.DataFrame(df_3m['CustomerID'].unique())
df_user.columns = ['CustomerID']
df_user.head()

CustomerID
0     14620.0
1     14740.0
2     13880.0
3     16462.0
4     17068.0

## (1) Recency Score

In [7]:
df_max_purchase = df_3m.groupby('CustomerID').InvoiceDate.max().reset_index()
df_max_purchase.columns = ['CustomerID','MaxPurchaseDate']
df_max_purchase.head()

CustomerID     MaxPurchaseDate
0     12747.0 2011-05-25 09:57:00
1     12748.0 2011-05-31 12:50:00
2     12749.0 2011-05-23 17:08:00
3     12821.0 2011-05-09 15:51:00
4     12823.0 2011-03-30 10:36:00

In [8]:
df_max_purchase['Recency'] = (df_max_purchase['MaxPurchaseDate'].max() - df_max_purchase['MaxPurchaseDate']).dt.days
df_user = pd.merge(df_user, df_max_purchase[['CustomerID','Recency']], on='CustomerID')
df_user.head()

CustomerID  Recency
0     14620.0       12
1     14740.0        4
2     13880.0       25
3     16462.0       91
4     17068.0       11

In [9]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_user[['Recency']])
df_user['RecencyCluster'] = kmeans.predict(df_user[['Recency']])
df_user.head()

CustomerID  Recency  RecencyCluster
0     14620.0       12               2
1     14740.0        4               2
2     13880.0       25               0
3     16462.0       91               1
4     17068.0       11               2

In [10]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [11]:
df_user = order_cluster('RecencyCluster', 'Recency',df_user,False)
df_user.head()

CustomerID  Recency  RecencyCluster
0     14620.0       12               3
1     14740.0        4               3
2     17068.0       11               3
3     12971.0        4               3
4     15194.0        6               3

## (2) Frequency Score

In [12]:
df_frequency = df_3m.groupby('CustomerID').InvoiceDate.count().reset_index()
df_frequency.columns = ['CustomerID','Frequency']
df_user = pd.merge(df_user, df_frequency, on='CustomerID')
df_frequency.head()

CustomerID  Frequency
0     12747.0         35
1     12748.0        582
2     12749.0         54
3     12821.0          6
4     12823.0          1

In [13]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_user[['Frequency']])
df_user['FrequencyCluster'] = kmeans.predict(df_user[['Frequency']])

df_user = order_cluster('FrequencyCluster', 'Frequency',df_user,True)
df_user.head()

CustomerID  Recency  RecencyCluster  Frequency  FrequencyCluster
0     14620.0       12               3         30                 0
1     15194.0        6               3         64                 0
2     18044.0        5               3         57                 0
3     18075.0       12               3         35                 0
4     15241.0        0               3         64                 0

### (3) Revenue = UnitPrice * Quantity

In [14]:
df_3m['Revenue'] = df_3m['UnitPrice'] * df_3m['Quantity']
df_revenue = df_3m.groupby('CustomerID').Revenue.sum().reset_index()
df_user = pd.merge(df_user, df_revenue, on='CustomerID')

In [15]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_user[['Revenue']])
df_user['RevenueCluster'] = kmeans.predict(df_user[['Revenue']])
df_user = order_cluster('RevenueCluster', 'Revenue',df_user,True)
df_user.head()

CustomerID  Recency  RecencyCluster  Frequency  FrequencyCluster  Revenue  \
0     14620.0       12               3         30                 0   393.28   
1     15194.0        6               3         64                 0  1439.02   
2     18044.0        5               3         57                 0   808.96   
3     18075.0       12               3         35                 0   638.12   
4     15241.0        0               3         64                 0   947.55   

   RevenueCluster  
0               0  
1               0  
2               0  
3               0  
4               0

In [16]:
df_user['OverallScore'] = df_user['RecencyCluster'] + df_user['FrequencyCluster'] + df_user['RevenueCluster']
df_user['Segment'] = 'Low-Value'
df_user.loc[df_user['OverallScore']>2,'Segment'] = 'Mid-Value' 
df_user.loc[df_user['OverallScore']>4,'Segment'] = 'High-Value'

In [17]:
df_user.head()

CustomerID  Recency  RecencyCluster  Frequency  FrequencyCluster  Revenue  \
0     14620.0       12               3         30                 0   393.28   
1     15194.0        6               3         64                 0  1439.02   
2     18044.0        5               3         57                 0   808.96   
3     18075.0       12               3         35                 0   638.12   
4     15241.0        0               3         64                 0   947.55   

   RevenueCluster  OverallScore    Segment  
0               0             3  Mid-Value  
1               0             3  Mid-Value  
2               0             3  Mid-Value  
3               0             3  Mid-Value  
4               0             3  Mid-Value

### Now, for the 6 month data, there is no cost specified in the dataset. That’s why Revenue becomes our LTV directly.

In [18]:
df_6m['Revenue'] = df_6m['UnitPrice'] * df_6m['Quantity']
df_user_6m = df_6m.groupby('CustomerID')['Revenue'].sum().reset_index()
df_user_6m.columns = ['CustomerID','m6_Revenue']


#plot LTV histogram
plot_data = [
    go.Histogram(
        x=df_user_6m.query('m6_Revenue < 10000')['m6_Revenue']
    )
]

plot_layout = go.Layout(
        title='6m Revenue'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

### Merging 3months and 6months data and plot its scatterplot.

In [19]:
df_merge = pd.merge(df_user, df_user_6m, on='CustomerID', how='left')
df_merge = df_merge.fillna(0)

df_graph = df_merge.query("m6_Revenue < 30000")

plot_data = [
    go.Scatter(
        x=df_graph.query("Segment == 'Low-Value'")['OverallScore'],
        y=df_graph.query("Segment == 'Low-Value'")['m6_Revenue'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=df_graph.query("Segment == 'Mid-Value'")['OverallScore'],
        y=df_graph.query("Segment == 'Mid-Value'")['m6_Revenue'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=df_graph.query("Segment == 'High-Value'")['OverallScore'],
        y=df_graph.query("Segment == 'High-Value'")['m6_Revenue'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "6m LTV"},
        xaxis= {'title': "RFM Score"},
        title='LTV'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [20]:
#remove outliers
df_merge = df_merge[df_merge['m6_Revenue']<df_merge['m6_Revenue'].quantile(0.99)]

#creating 3 clusters
kmeans = KMeans(n_clusters=3)
kmeans.fit(df_merge[['m6_Revenue']])
df_merge['LTVCluster'] = kmeans.predict(df_merge[['m6_Revenue']])

#order cluster number based on LTV
df_merge = order_cluster('LTVCluster', 'm6_Revenue',df_merge,True)

#creatinga new cluster dataframe
df_cluster = df_merge.copy()

#see details of the clusters
df_cluster.groupby('LTVCluster')['m6_Revenue'].describe()

count         mean          std      min       25%       50%  \
LTVCluster                                                                  
0           1394.0   396.137189   419.891843  -609.40     0.000   294.220   
1            371.0  2492.794933   937.341566  1445.31  1731.980  2162.930   
2             56.0  8222.565893  2983.572030  5396.44  6151.435  6986.545   

                  75%       max  
LTVCluster                       
0            682.4300   1429.87  
1           3041.9550   5287.39  
2           9607.3225  16756.31

### Do one-hot encoding on segment clusters for training.

In [24]:
#convert categorical columns to numerical
df_class = pd.get_dummies(df_cluster)

#calculate and show correlations
corr_matrix = df_class.corr()
corr_matrix['LTVCluster'].sort_values(ascending=False)

#create X and y, X will be feature set and y is the label - LTV
X = df_class.drop(['LTVCluster','m6_Revenue'],axis=1)
y = df_class['LTVCluster']

#split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=56)
X.head()

CustomerID  Recency  RecencyCluster  Frequency  FrequencyCluster  Revenue  \
0     14620.0       12               3         30                 0   393.28   
1     18044.0        5               3         57                 0   808.96   
2     18075.0       12               3         35                 0   638.12   
3     15241.0        0               3         64                 0   947.55   
4     15660.0        4               3         34                 0   484.62   

   RevenueCluster  OverallScore  Segment_High-Value  Segment_Low-Value  \
0               0             3                   0                  0   
1               0             3                   0                  0   
2               0             3                   0                  0   
3               0             3                   0                  0   
4               0             3                   0                  0   

   Segment_Mid-Value  
0                  1  
1                  1  
2                  1  
3                  1  
4                  1

In [22]:
ltv_xgb_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1,objective= 'multi:softprob',n_jobs=-1).fit(X_train, y_train)

print('Accuracy of XGB classifier on training set: {:.2f}'
       .format(ltv_xgb_model.score(X_train, y_train)))
print('Accuracy of XGB classifier on test set: {:.2f}'
       .format(ltv_xgb_model.score(X_test[X_train.columns], y_test)))

y_pred = ltv_xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

Accuracy of XGB classifier on training set: 0.90
Accuracy of XGB classifier on test set: 0.87
              precision    recall  f1-score   support

           0       0.90      0.99      0.94        70
           1       0.82      0.50      0.62        18
           2       0.50      0.50      0.50         4

   micro avg       0.87      0.87      0.87        92
   macro avg       0.74      0.66      0.69        92
weighted avg       0.86      0.87      0.86        92

